In [ ]:
import copy
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import time
import re
import itertools
import sys
import pickle
import os
import string
import autoreload

from random import shuffle
from IPython.display import clear_output, display, Image, HTML
from scipy.spatial.distance import cosine

sys.path.append('../neural_programmer')

from notebook_utils import init_data, build_graph, restore_model
from neural_programmer import evaluate

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
DATA_DIR = '/scratch/pramodkm/neural_programmer/wtq_data_orig/data/'
MODEL_FILE = os.path.join('/home/pramodkm/interpreting_np/trained_model', 'model_92500')

In [ ]:
train_data, dev_data, test_data, utility = init_data(DATA_DIR)

In [ ]:
tf.reset_default_graph()
with tf.device('/device:GPU:1'):
    sess, graph, params = build_graph(utility)

In [ ]:
sess, graph = restore_model(sess, graph, params, MODEL_FILE)

In [ ]:
evaluate(sess, dev_data, utility.FLAGS.batch_size, graph, 92500)